In [1]:
# flickr example

In [43]:
import requests
from lxml import html
import os

In [3]:
def prepare_query(q):
    replace_dict = {" ": "%20"}
    for k, v in replace_dict.items():
        q = q.replace(k, v)
    return q

In [53]:
qlist = ['КПРФ',
        'ЛДПР',
        'Гражданская инициатива',
        'партия Яблоко',
        'Российский общенародный союз',
        'Партия роста',
        'Коммунисты России']

target_pat = "https://www.flickr.com/search/?text={}&sort=date-posted-desc&view_all=1"
target_list = []
for q in qlist:
    target = target_pat.format(prepare_query(q))
    print(target)
    target_list.append(target)

https://www.flickr.com/search/?text=КПРФ&sort=date-posted-desc&view_all=1
https://www.flickr.com/search/?text=ЛДПР&sort=date-posted-desc&view_all=1
https://www.flickr.com/search/?text=Гражданская%20инициатива&sort=date-posted-desc&view_all=1
https://www.flickr.com/search/?text=партия%20Яблоко&sort=date-posted-desc&view_all=1
https://www.flickr.com/search/?text=Российский%20общенародный%20союз&sort=date-posted-desc&view_all=1
https://www.flickr.com/search/?text=Партия%20роста&sort=date-posted-desc&view_all=1
https://www.flickr.com/search/?text=Коммунисты%20России&sort=date-posted-desc&view_all=1


In [55]:
def get_tree(target):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    response = requests.get(target, headers=headers) # get response
    tree = html.fromstring(response.content) # convert content text to lxml obj
    return tree
tree_list = [get_tree(target) for target in target_list]
tree_list

[<Element html at 0xdd6a813868>,
 <Element html at 0xdd6a345548>,
 <Element html at 0xdd6a345b88>,
 <Element html at 0xdd6a345f48>,
 <Element html at 0xdd6a813a48>,
 <Element html at 0xdd6a813188>,
 <Element html at 0xdd6a7a1f98>]

In [56]:
test_target = target_list[0]
test_tree = tree_list[0]
print(test_target, test_tree)

https://www.flickr.com/search/?text=КПРФ&sort=date-posted-desc&view_all=1 <Element html at 0xdd6a813868>


In [57]:
def get_link_from_style(style_att):
    _, link = style_att[:-1].split("url(//")
    return "https://" + link

In [58]:
def savefile_stream(link, ofile):
    link = link.replace("http://", "https://")
    response = requests.get(link, stream=True)
    chunk_size = 20000
    if response.status_code == 200:
        with open(ofile, 'wb') as fd:
            for chunk in response.iter_content(chunk_size):
                fd.write(chunk)
    else:
        print("downloading failed")

In [59]:
def get_photo_tree(tree):
    photo_div_view_class = "view search-photos-everyone-view requiredToShowOnServer"
    photo_div_class = "view photo-list-photo-view requiredToShowOnServer awake"
    photo_links = []
    for photo_div in tree.find_class(photo_div_class):
        cur_style = photo_div.attrib.get("style")
        photo_link = get_link_from_style(cur_style)
        photo_links.append(photo_link)
    return photo_links

photo_links = get_photo_tree(test_tree)
for photo_link in photo_links:
    print(photo_link)
    savefile_stream(photo_link, "img/test/"+photo_link.split("/")[-1])

https://c1.staticflickr.com/5/4651/25771010467_b404b662fd.jpg
https://c1.staticflickr.com/5/4785/38817611910_5fec150938_n.jpg
https://c1.staticflickr.com/5/4709/39917311934_66c2828a57_n.jpg
https://c1.staticflickr.com/5/4791/25703961297_ab9b2cbbd2.jpg
https://c1.staticflickr.com/5/4626/39860162704_ceb60a0dda.jpg
https://c1.staticflickr.com/5/4702/38728451650_e5b965b88d.jpg
https://c1.staticflickr.com/5/4698/39805856804_34de1837f0.jpg
https://c1.staticflickr.com/5/4715/40114710221_9e995b7c21_n.jpg
https://c1.staticflickr.com/5/4709/28330501119_8fedf6726e_n.jpg
https://c1.staticflickr.com/5/4612/28318644899_05cf46defc_n.jpg
https://c1.staticflickr.com/5/4756/39381750364_2140537454_n.jpg
https://c1.staticflickr.com/5/4708/26199754968_3dcfe1303d_n.jpg
https://c1.staticflickr.com/5/4758/39161641905_c93f1d0004.jpg
https://c1.staticflickr.com/5/4766/26182665058_3df831cde1_m.jpg
https://c1.staticflickr.com/5/4708/28199940899_aee1eee399_n.jpg
https://c1.staticflickr.com/5/4669/39145570424_6e751

In [60]:
odir = "img/"
for q, tree in zip(qlist, tree_list):
    qdir = odir+q+"/"
    if not os.path.exists(qdir):
        os.mkdir(qdir)
    print(qdir)
    photo_links = get_photo_tree(tree)
    for photo_link in photo_links:
        print(photo_link)
        savefile_stream(photo_link, qdir + photo_link.split("/")[-1])

img/КПРФ/
https://c1.staticflickr.com/5/4651/25771010467_b404b662fd.jpg
https://c1.staticflickr.com/5/4785/38817611910_5fec150938_n.jpg
https://c1.staticflickr.com/5/4709/39917311934_66c2828a57_n.jpg
https://c1.staticflickr.com/5/4791/25703961297_ab9b2cbbd2.jpg
https://c1.staticflickr.com/5/4626/39860162704_ceb60a0dda.jpg
https://c1.staticflickr.com/5/4702/38728451650_e5b965b88d.jpg
https://c1.staticflickr.com/5/4698/39805856804_34de1837f0.jpg
https://c1.staticflickr.com/5/4715/40114710221_9e995b7c21_n.jpg
https://c1.staticflickr.com/5/4709/28330501119_8fedf6726e_n.jpg
https://c1.staticflickr.com/5/4612/28318644899_05cf46defc_n.jpg
https://c1.staticflickr.com/5/4756/39381750364_2140537454_n.jpg
https://c1.staticflickr.com/5/4708/26199754968_3dcfe1303d_n.jpg
https://c1.staticflickr.com/5/4758/39161641905_c93f1d0004.jpg
https://c1.staticflickr.com/5/4766/26182665058_3df831cde1_m.jpg
https://c1.staticflickr.com/5/4708/28199940899_aee1eee399_n.jpg
https://c1.staticflickr.com/5/4669/3914557

In [62]:
# to be continue